In [1]:
import os
import sys
# 将项目根目录（MyHub）的父目录添加到系统路径中
# 这使得 'MyHub' 本身可以被当作一个包来导入
project_root = os.path.abspath(os.path.join(os.getcwd(), '../..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

import json
import time
import argparse
import datetime
import numpy as np
from pathlib import Path
import timm.optim.optim_factory as optim_factory
from torch.utils.tensorboard.writer import SummaryWriter

from MyHub.core.registry import DATASETS, MODELS, POSTFUNCS, TRANSFORMS, EVALUATORS, build_from_registry
from MyHub.training_scripts.utils import misc
from MyHub.training_scripts.utils.yaml import load_yaml_config,split_config, add_attr
from MyHub.training_scripts.trainer import train_one_epoch
from MyHub.training_scripts.tester import test_one_epoch

/home/chengxiaozhen/anaconda3/envs/forensichub2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2.1.2
Current Torch version: 2.1


In [2]:
parser = argparse.ArgumentParser(description='Training Script')
parser.add_argument('--config', default='config/resnet_train.yaml', help='path to config file', type=str)

args = parser.parse_args(['--config', '/home/chengxiaozhen/Test/ForensicHub/MyHub/config/resnet_train.yaml'])

config = load_yaml_config(args.config)
args, model_args, train_dataset_args, test_dataset_args, transform_args, evaluator_args = split_config(config)
add_attr(args,output_dir=args.log_dir)
add_attr(args,if_not_amp=not args.use_amp)

if not os.path.exists(args.output_dir):
    os.makedirs(args.output_dir, exist_ok=True)

In [3]:
print("Config loaded:")
print("args:", args)
print("model_args:", model_args)
print("train_dataset_args:", train_dataset_args)
print("test_dataset_args:", test_dataset_args)
print("transform_args:", transform_args)
print("evaluator_args:", evaluator_args)

Config loaded:
args: Namespace(gpus='0,1,2,3', flag='train', log_dir='./log/resnet_train', if_predict_label=True, if_predict_mask=False, batch_size=512, test_batch_size=128, epochs=20, accum_iter=1, record_epoch=20, no_model_eval=False, test_period=1, log_per_epoch_count=20, find_unused_parameters=True, use_amp=True, weight_decay=0.05, lr=0.0001, blr=0.001, min_lr=1e-05, warmup_epochs=1, device='cuda', seed=42, resume='', start_epoch=0, num_workers=8, pin_mem=True, world_size=1, local_rank=-1, dist_on_itp=False, dist_url='env://', output_dir='./log/resnet_train', if_not_amp=False)
model_args: {'name': 'Resnet101', 'init_config': {'output_type': 'label', 'image_size': 256}}
train_dataset_args: {'name': 'CrossDataset', 'dataset_name': 'CrossDataset', 'init_config': {'dataset_config': [{'name': 'IMDLCrossDataset', 'pic_nums': 12641, 'init_config': {'image_size': 256, 'path': '/mnt/data0/public_datasets/IML/CASIAv2_with_label.json'}}, {'name': 'AIGCCrossDataset', 'pic_nums': 12641, 'init_c